In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od

In [3]:
help(od.download)

Help on function download in module opendatasets:

download(dataset_id_or_url, data_dir='.', force=False, dry_run=False, **kwargs)



In [4]:
import os
os.listdir("C://Users")

['All Users', 'Default', 'Default User', 'desktop.ini', 'pooja', 'Public']

In [5]:
dataset = 'https://www.kaggle.com/datasets/rsrishav/youtube-trending-video-dataset?select=IN_youtube_trending_data.csv'

In [6]:
od.download(dataset, data_dir='C:\Users\pooja\Downloads')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (Temp/ipykernel_7264/4036904024.py, line 1)

In [ ]:
import os

In [ ]:
data_dir = '.\youtube-trending-video-dataset'

In [ ]:
os.listdir(data_dir)

In [ ]:
import pandas as pd

In [ ]:
life_df = pd.read_csv('IN_youtube_trending_data.csv')
life_df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('C://Users//timbe//Downloads//IN_youtube_trending_data.csv')

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.head(3)

In [ ]:
data.drop(['video_id', 'channelId'], axis = 1, inplace = True)

In [ ]:
data.head(3)

In [ ]:
# Checking null values
data.isna().sum()

In [ ]:
data.info()

In [ ]:
# Converting 'publishedAt' and 'trending_date' to datetime objects

data['publishedAt'] = pd.to_datetime(data['publishedAt'])
data['trending_date'] = pd.to_datetime(data['trending_date'])

In [ ]:
# Validating the above changes

data.dtypes[['publishedAt', 'trending_date']]

In [ ]:
# Getting new column 'daysTakenToTrend' which gives us the information about the number of days taken by the video to get into the trending videos

data['daysTakenToTrend'] = (data['trending_date'] - data['publishedAt']).dt.days

# Validating the above code

data.head()

In [ ]:
sns.histplot(data['daysTakenToTrend'], kde = True, bins = 20)

From the above plot, 

## EDA